### 실습 개요
- 당신은 App 개발자입니다.
- 당신은 영상을 `역실행(backwards_video)`, `색상반전(reverse_video)` 시키는 함수들을 만들어 사용자에게 제공해야 합니다. 또한 향후 다양한 영상 편집 함수를 추가로 만들어야 합니다.
- 이해하기 쉽고, 생산성이 높은 구현 방법을 찾으세요.
- video 입출력 및 공통 기능 개발은 Library 개발자가 대신 할 것입니다. 그를 잘 활용하세요.

In [ ]:
import numpy as np

# functions.py 에 미리 정의된 함수들을 import 한다
from functions import video_2_ndarray, ndarray_2_video

### 1. library의 Video 함수들를 직접 사용하기
- 가장 기본적인 방식.
- “공통 로직”을 중복해서 호출하는 불편함
- “App 개발자”가 `video_2_ndarray`, `ndarray_2_video`등 어려운 함수들을 정확하게 파악하고 사용해야 함

In [ ]:
# App 개발자 영역 : library의 video_2_ndarray, ndarray_2_video 등 전문 영역 함수들의 사용법을 알아야 한다
def backwards(video_array: np.ndarray) -> np.ndarray:
    return video_array[::-1]

def reverse(video_array: np.ndarray) -> np.ndarray:
    return 255 - video_array

def backwards_video(input_path: str, output_path: str):

    video_array, _, _ = video_2_ndarray(input_path)
    modified_video = backwards(video_array)
    ndarray_2_video(modified_video, output_path)

def reverse_video(input_path: str, output_path: str):

    video_array, _, _ = video_2_ndarray(input_path)
    modified_video = reverse(video_array)
    ndarray_2_video(modified_video, output_path)

# 사용자 영역
backwards_video('../media/SampleVideo_640x360_5mb.mp4', 'backwards_video_1.mp4')
reverse_video('../media/SampleVideo_640x360_5mb.mp4', 'reverse_video_1.mp4')

### 2. Callback 함수 사용하기
- “App 개발자”가 `video_2_ndarray`, `ndarray_2_video`를 알 필요없음
- “App 개발자”가 Callback 함수 개념을 알고 구현해야 함

In [ ]:
from typing import Callable

# library 개발자 영역
def modify_video(input_path: str, output_path: str, func: Callable):

    video_array, _, _ = video_2_ndarray(input_path)    
    modified_video = func(video_array)
    ndarray_2_video(modified_video, output_path)

# App 개발자 영역 : Callback 함수 사용법 알아야 함함 
def backwards(video_array: np.ndarray) -> np.ndarray:
    return video_array[::-1]

def reverse(video_array: np.ndarray) -> np.ndarray:
    return 255 - video_array

def backwards_video(input_path: str, output_path: str):
    return modify_video(input_path, output_path, backwards) 

def reverse_video(input_path: str, output_path: str):
    return modify_video(input_path, output_path, reverse) 

# 사용자 영역
backwards_video('../media/SampleVideo_640x360_5mb.mp4', 'backwards_video_2.mp4') 
reverse_video('../media/SampleVideo_640x360_5mb.mp4', 'reverse_video_2.mp4') 

### 3. 함수를 return 하는 Closure 사용하기
- “App 개발자”가 함수가 함수의 인자로 그리고 return 값으로 사용되는 과정을 이해해야 함

In [ ]:
from typing import Callable

# library 개발자 영역
def modify_video(func: Callable) -> Callable:
    def wrapper(input_path: str, output_path: str):

        video_array, _, _ = video_2_ndarray(input_path)
        modified_video = func(video_array)
        ndarray_2_video(modified_video, output_path)

    return wrapper

# App 개발자 영역 : Closure 함수 사용법 알아야 함
def backwards(video_array: np.ndarray) -> np.ndarray:
    return video_array[::-1]

def reverse(video_array: np.ndarray) -> np.ndarray:
    return 255 - video_array

backwards_video = modify_video(backwards)
reverse_video = modify_video(reverse)

# 사용자 영역
backwards_video('../media/SampleVideo_640x360_5mb.mp4', 'backwards_video_3.mp4')
reverse_video('../media/SampleVideo_640x360_5mb.mp4', 'reverse_video_3.mp4')

### 4. Decorator 사용하기
- 함수가 인자나 return값으로 넘나드는 과정을 은닉하고 추상화하여 직관적이고 쉽게 표현

In [ ]:
# library 개발자 영역
def modify_video(func: Callable) -> Callable:
    def wrapper(input_path: str, output_path: str):

        video_array, _, _ = video_2_ndarray(input_path)
        modified_video = func(video_array)
        ndarray_2_video(modified_video, output_path)

    return wrapper

# App 개발자 영역
@modify_video
def backwards_video(video_array: np.ndarray) -> np.ndarray:
    return video_array[::-1]

@modify_video
def reverse_video(video_array: np.ndarray) -> np.ndarray:
    return 255 - video_array

# 사용자 영역
backwards_video('../media/SampleVideo_640x360_5mb.mp4', 'backwards_video_4.mp4')
reverse_video('../media/SampleVideo_640x360_5mb.mp4', 'reverse_video_4.mp4')

### 따라 해보기 #1. 
- `backwards_video`를 Web UI로 구현하도록 library 개발자(genAI 서비스)에게 요청하기

In [ ]:
"""
 prompt : 

    다음 code는 mp4 파일 input_path를 받아서
    func에 의해 영상을 가공한 후
    output_path에 다시 mp4를 저장하는 절차다.
    video_2_ndarray, ndarray_2_video 는 이미 만들어져 있다.
    ```
    video_array, _, _ = video_2_ndarray(input_path)
    modified_video = func(video_array)
    ndarray_2_video(modified_video, output_path)
    ```

    아래 code를 실행했을 때
    - gradio UI 가 나타남 
    - UI상에서 input_path를 입력받고
    - submit 실행하면 backwards_video 함수에 의해 변환된 동영상이 play되도록 serve_modified_video 를 구현해줘
    ---
    @serve_modified_video
    def backwards_video(video_array):
        return video_array[::-1]

    backwards_video()
    
"""

In [ ]:
# serve_modified_video 를 functions.py에 저장하고 import
from functions import serve_modified_video

@serve_modified_video
def backwards_video(video_array: np.ndarray) -> np.ndarray:
    return video_array[::-1]

# 사용자 영역
backwards_video()